In [376]:
import torch

In [377]:
x = torch.arange(4.0)

Avoid allocating new memory every time we take a derivative because deep learning requires successively computing derivatives with respect to the same parameters a great many times, and we might risk running out of memory


In [378]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad  # The gradient is None by default

In [379]:
x

tensor([0., 1., 2., 3.], requires_grad=True)

Differentiating the function y = 2x@x with respect to the column vector x. To start, we assign x an initial value.

In [380]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [381]:
y.backward() # take the gradient of y with respect to x;  derivative of 2x@x is 4x
x.grad

tensor([ 0.,  4.,  8., 12.])

In [382]:
x.grad == 4 * x

tensor([True, True, True, True])

Note that PyTorch does not automatically reset the gradient buffer when we record a new gradient; the new gradient is added to the already-stored gradient. This behavior comes in handy when we want to optimize the sum of multiple objective functions.


In [383]:
x.grad

tensor([ 0.,  4.,  8., 12.])

In [384]:
x.grad.zero_()  # Reset the gradient
x.grad, y

(tensor([0., 0., 0., 0.]), tensor(28., grad_fn=<MulBackward0>))

You have a tensor $Y$, which has been computed directly or indirectly from tensor $X$.

`Y.backward()` would calculate the derivative of each element of $Y$ w.r.t. each element of $X$. This gives us `N_out` (the number of elements in Y) masks with shape `X.shape`.

However, `torch.backward()` enforces by default that the gradient that will be stored in `X.grad` shall be of the same shape as X. If `N_out=1`, there is no problem as we have only one mask. That is why you want to reduce Y to a single value...

`some_loss_function.sum().backward()` calculates the sum of all the loss values across the batch and then performs backpropagation based on that sum. This means that each element of the batch contributes equally to the loss, regardless of its value. This can be useful in some scenarios, such as when you want to prioritize rare events that have a small number of occurrences in the batch. If you sum your loss you will end up scaling your loss value and the gradients that are inferred from it uncontrollably -> overflow after some time.

`some_loss_function.mean().backward()` calculates the mean of all the loss values across the batch and then performs backpropagation based on that mean. This means that each element of the batch contributes equally to the loss, but the contribution is weighted by its value. This can be useful in scenarios where you want to prioritize elements of the batch that have a higher loss value, or when you want to ensure that the gradients are scaled appropriately.

If `N_out>1`, Pytorch wants to take a weighted sum over the `N_out` gradient masks. But you need to supply the weights for this weighted sum! You can do this with the gradient argument:
`Y.backward(gradient=weights_shaped_like_Y)`

If you give every element of Y weight 1, you will get the same behaviour as using `torch.sum(Y).backward()`


https://zhang-yang.medium.com/the-gradient-argument-in-pytorchs-backward-function-explained-by-examples-68f266950c29

NOTE: We did not pass the ``gradient`` argument to ``backward()``, and this defaults to passing the value 1. PyTorch is calculating the Jacobian product. In the case of scalar values, ``.backward()`` w/o parameters is equivalent to ``.backward(torch.tensor(1.0))``. When input is a vector and output y = x1 + x2 is a scalar, the default gradient argument will also be 1s. If output is a vector then have something like y1 = f(x1) and y2 = f(x2), etc.

In [385]:
# You can only compute partial derivatives for a *scalar* function. 
# What backwards() gives you is d(loss)/d(parameter) and you expect 
# a *single* gradient value per parameter.
y = x.sum()
y.backward(), x.grad

(None, tensor([1., 1., 1., 1.]))

In [386]:
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [387]:
x.grad

tensor([1., 1., 1., 1.])

In [388]:
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

Detaching computational graph
https://www.youtube.com/watch?v=hjnVLfvhN0Q

In [389]:
x.grad.zero_()
y = x * x
u = y.detach() # assumes u no dependency on x even if elements were x**2
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [390]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [391]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [392]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

Even though our function ``f`` is, for demonstration purposes, a bit contrived, its dependence on the input is quite simple: it is a linear function of a with piecewise defined scale. As such, ``f(a) / a`` is a vector of constant entries and, moreover, ``f(a) / a`` needs to match the gradient of ``f(a)`` with respect to ``a``. Note that the returned $c \propto a(2^{n+1})$ so that the gradient with respect to input is the same as dividing out the variable in this case.

In [393]:
a.grad, a, d

(tensor(8192.),
 tensor(0.1720, requires_grad=True),
 tensor(1408.6375, grad_fn=<MulBackward0>))

In [394]:
a.grad == d / a

tensor(True)

More insight on computational graphs in PyTorch: https://www.youtube.com/watch?v=MswxJw-8PvE

More insight on hooks (backward hooks fixed) in Pytorch: https://www.youtube.com/watch?v=syLFCVYua6Q